## Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
- The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [5]:
import mlflow

In [6]:
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

In [7]:
print(f'The tracking URI is {mlflow.get_tracking_uri()} ')

The tracking URI is http://127.0.0.1:5000/ 


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716233294528, experiment_id='0', last_update_time=1716233294528, lifecycle_stage='active', name='Default', tags={}>]

## train an experiment 

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment('scenario 2 ')

with mlflow.start_run():
    
    X,y = load_iris(return_X_y=True)
    params = {"n_estimators": 1000, "random_state": 42}

    mlflow.log_params(params)
    
    model = RandomForestClassifier(**params).fit(X,y)
    y_pred = model.predict(X)
    
    acc = accuracy_score(y, y_pred)
    mlflow.log_metric('accuracy', acc)
    mlflow.sklearn.log_model(model, artifact_path='models')
    print(f'default artifact path is {mlflow.get_artifact_uri()}')
    

2024/05/20 20:32:51 INFO mlflow.tracking.fluent: Experiment with name 'scenario 2 ' does not exist. Creating a new experiment.


default artifact path is /Users/davidgillespie/projects/mlops_course/week 2/course_work/scenarios/local_artifacts/1/e43f819f7cec4e4b9d6225db05abee93/artifacts


In [13]:
from mlflow.tracking import MlflowClient

client = MlflowClient(tracking_uri = 'http://127.0.0.1:5000/')

In [14]:
client.search_registered_models()

[]

In [62]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

In [64]:
mlflow.register_model(model_uri=f'runs:/{run_id}/models',
                      name='iris-classifier')

Successfully registered model 'iris-classifier'.
2024/05/20 20:52:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716234760800, current_stage='None', description='', last_updated_timestamp=1716234760800, name='iris-classifier', run_id='e43f819f7cec4e4b9d6225db05abee93', run_link='', source=('/Users/davidgillespie/projects/mlops_course/week '
 '2/course_work/scenarios/local_artifacts/1/e43f819f7cec4e4b9d6225db05abee93/artifacts/models'), status='READY', status_message='', tags={}, user_id='', version='1'>